# SLM Deep Researcher Example

This notebook demonstrates streaming research with the slm-deep-researcher API.

In [1]:
import os
import json
import requests
from typing import Iterator, Dict, Any
from dataclasses import dataclass

from rich.console import Console
from rich.panel import Panel
from rich.markdown import Markdown

In [2]:
console = Console()
BASE_URL = os.environ.get('LANGGRAPH_BASE_URL', 'http://127.0.0.1:2024')

In [3]:
@dataclass
class ResearchProgress:
    stage: str
    query: str = None
    sources: list[str] = None
    summary: str = None
    is_final: bool = False

In [4]:
class ResearchClient:
    def __init__(self, base_url: str = BASE_URL):
        self.base_url = base_url
        self.headers = {'Content-Type': 'application/json'}
        self.assistant_id = None
        self._ensure_connection()
        self._create_assistant()

    def _ensure_connection(self):
        try:
            response = requests.get(f'{self.base_url}/ok', timeout=5)
            response.raise_for_status()
        except requests.RequestException as e:
            raise ConnectionError(f"Cannot connect to LangGraph server: {e}")

    def _create_assistant(self):
        payload = {
            'name': 'slm-deep-researcher',
            'graph_id': 'slm_deep_researcher',
            'if_exists': 'do_nothing',
        }
        response = requests.post(
            f'{self.base_url}/assistants',
            headers=self.headers,
            json=payload,
            timeout=10
        )
        response.raise_for_status()
        self.assistant_id = response.json()['assistant_id']

    def research(self, topic: str) -> Iterator[ResearchProgress]:
        payload = {
            "assistant_id": self.assistant_id,
            "input": {"research_topic": topic},
            "on_completion": "delete",
            "stream_mode": ["events"],
        }

        with requests.post(
            f"{self.base_url}/runs/stream",
            json=payload,
            stream=True,
            timeout=300
        ) as resp:
            resp.raise_for_status()

            for raw in resp.iter_lines():
                if not raw:
                    continue

                text = raw.decode()
                if not text.startswith("data:"):
                    continue

                event = json.loads(text[5:])
                progress = self._parse_event(event)
                if progress:
                    yield progress

    def _parse_event(self, event: Dict[str, Any]) -> ResearchProgress:
        kind = event.get("event")
        name = event.get("name")
        data = event.get("data", {})

        if kind != "on_chain_end" or not name:
            return None

        output = data.get("output", {})

        if name == "generate_query":
            query = output.get("search_query")
            if query:
                return ResearchProgress(stage="query_generated", query=query)

        elif name == "web_research":
            sources = output.get("sources_gathered")
            if sources and sources[-1]:
                source_list = [
                    line.strip() for line in sources[-1].split("\n")
                    if line.startswith("* ")
                ]
                return ResearchProgress(stage="sources_found", sources=source_list)

        elif name == "summarize_sources":
            summary = output.get("running_summary")
            if summary:
                return ResearchProgress(stage="summary_updated", summary=summary)

        elif name == "reflect_on_summary":
            query = output.get("search_query")
            if query:
                return ResearchProgress(stage="reflection_query", query=query)

        elif name == "finalize_summary":
            summary = output.get("running_summary")
            if summary:
                return ResearchProgress(stage="completed", summary=summary, is_final=True)

        return None

In [5]:
def display_progress(progress: ResearchProgress, loop_count: dict):
    if progress.stage == "query_generated":
        console.print(f"\n[bold cyan]🔍 Initial Query[/bold cyan]")
        console.print(f"[cyan]   {progress.query}[/cyan]")

    elif progress.stage == "sources_found":
        console.print(f"[bold green]✓[/bold green] [green]Found {len(progress.sources)} sources[/green]")
        for source in progress.sources[:3]:
            source_clean = source.lstrip("* ")
            console.print(f"   [dim]{source_clean}[/dim]")

    elif progress.stage == "summary_updated":
        preview = progress.summary.replace("\n", " ")[:100]
        console.print(f"[bold yellow]📝[/bold yellow] [yellow]Summary updated[/yellow]")
        console.print(f"   [dim]{preview}...[/dim]")

    elif progress.stage == "reflection_query":
        loop_count['value'] += 1
        console.print(f"\n[bold magenta]🔍 Follow-up Query #{loop_count['value']}[/bold magenta]")
        console.print(f"[magenta]   {progress.query}[/magenta]")

    elif progress.stage == "completed":
        console.print("\n")
        console.rule("[bold green]Research Complete[/bold green]")
        console.print("\n")
        md = Markdown(progress.summary)
        console.print(md)

## Run Research

Set your research topic and run the cells below.

In [6]:
topic = "Investigate the environmental impact of training one large model vs. 10,000 tiny ones."

console.print(Panel.fit(
    f"[bold white]{topic}[/bold white]",
    title="[bold blue]Research Topic[/bold blue]",
    border_style="blue"
))
console.print()

╭──────────────────────────────────── Research Topic ────────────────────────────────────╮
│ Investigate the environmental impact of training one large model vs. 10,000 tiny ones. │
╰────────────────────────────────────────────────────────────────────────────────────────╯

In [7]:
client = ResearchClient()
loop_count = {'value': 0}

for progress in client.research(topic):
    display_progress(progress, loop_count)

🔍 Initial Query

   Environmental impact of training large vs. small models 2025

✓ Found 3 sources

Wikipedia Environmental impact of artificial intelligence - Wikipedia : 
https://en.wikipedia.org/wiki/Environmental_impact_of_artificial_intelligence

MIT News Explained: Generative AI’s environmental impact | MIT News | Massachusetts Institute of Technology : 
https://news.mit.edu/2025/explained-generative-ai-environmental-impact-0117

...

  Training large AI models like GPT-4 consumes significantly more energy than 10,000 tiny models, em...

🔍 Follow-up Query #4

   Implementations of energy-efficient training techniques for tiny AI models and their real-world carbon footprint
comparisons.

──────────────────────────────────────────────── Research Complete ────────────────────────────────────────────────

Summary                                                      

Training large AI models like GPT-4 consumes significantly more energy than 10,000 tiny models, emitting 626,000   
pounds of CO₂ equivalent for 300 round-trip flights between New York and San Francisco. While exact comparisons are
unclear, data suggests large models have higher carbon footprints due to their computational demands. Smaller      
models, such as Grok 3 Nano, demonstrate extreme energy efficiency, using 0.75% of battery power for 25            
conversations. The ECO-AI paper highlights that techniques like mixed-precision training and optimizer choices can 
reduce carbon footprints by over 30% with minimal performance loss, while model compression methods (e.g., pruning,
quantization) can cut energy use by 32.097% for BERT and 23.934% for ELECTRA. However, data-poisoning attacks, such
as injecting 250 malicious documents into training data, reveal that large models require near-constant poisoned   
content regardless of size, challenging assumptions about data scale. These findings underscore the need for       
sustainable practices, renewable integration, and policy measures to balance AI performance with environmental     
impact.                                                                                                            

                                                     Sources:                                                      

 • Wikipedia Environmental impact of artificial intelligence - Wikipedia :                                         
   https://en.wikipedia.org/wiki/Environmental_impact_of_artificial_intelligence                                   
 • MIT News Explained: Generative AI’s environmental impact | MIT News | Massachusetts Institute of Technology :   
   https://news.mit.edu/2025/explained-generative-ai-environmental-impact-0117                                     
 • The Sustainable Agency Environmental Impact of Generative AI | Stats & Facts for 2025 :                         
   https://thesustainableagency.com/blog/environmental-impact-of-generative-ai/                                    
 • Large language model - Wikipedia : https://en.wikipedia.org/wiki/Large_language_model                           
 • Tiny AI Model Shocks Industry by Outperforming Giants in ARC-AGI... :                                           
   https://www.theailibrary.co/blog/tiny-ai-model-shocks-industry-by-outperforming-giants-in-arc-agi-benchmark     
 • Introducing Gemma 3 270M: The compact model for hyper- efficient AI :                                           
   https://developers.googleblog.com/en/introducing-gemma-3-270m/                                                  
 • ECO-AI – Energy-Conscious Optimization for AI Training :                                                        
   https://openaccess.thecvf.com/content/CVPR2025W/ReGenAI/papers/Horvath_ECO-AI_-_Energy-Conscious_Optimization_fo
   r_AI_Training_CVPRW_2025_paper.pdf                                                                              
 • Sustainable AI: Measuring and Reducing Carbon Footprint in ... :                                                
   https://www.researchgate.net/publication/392727729_Sustainable_AI_Measuring_and_Reducing_Carbon_Footprint_in_Mod
   el_Training_and_Deployment                                                                                      
 • Comparative analysis of model compression techniques for ... :                                                  
   https://www.nature.com/articles/s41598-025-07821-w                                                              
 • GPT-3 - Wikipedia : https://en.wikipedia.org/wiki/GPT-3                                                         
 • Anthropic research on data-poisoning attacks in large language models :                                         
   https://www.anthropic.com/research/small-samples-poison                                                 